In [8]:
import numpy as np
import pandas as pd

# import and run file
from importnb import imports
with imports("ipynb"):
    import data_preparation
%run data_preparation.ipynb

In [9]:
# average rating for each restaurant
avg_scores = reviews.groupby(['Restaurant Name']).agg(count=('Rating', 'size'), avg_Rating=('Rating', 'mean')).reset_index()

# average shrinkage rating for each restaurant
rating_mean = reviews['Rating'].mean()
rating_count_mean = avg_scores['count'].mean()
avg_scores['shrinkage_Rating'] = (avg_scores['count'] * avg_scores['avg_Rating'] + rating_count_mean * rating_mean) / (avg_scores['count'] + rating_count_mean)



# match each restuarant with its average rating
data_res_scores = restaurants.merge(avg_scores, how='left', on='Restaurant Name')

In [10]:
# popularity matching recommendation system
def popularity_recommendation(cuisine_type, max_sugest, score_type='Default'):
    if (score_type == 'Shrinkage'):
        res_list = data_res_scores[data_res_scores['Cuisine'] == cuisine_type].sort_values(['shrinkage_Rating', 'count'], ascending=[False,False])
        return(res_list.loc[:,['Restaurant Name', 'count', 'avg_Rating', 'shrinkage_Rating']].head(max_sugest).reset_index(drop=True))
    else:
        res_list = data_res_scores[data_res_scores['Cuisine'] == cuisine_type].sort_values(['avg_Rating', 'count'], ascending=[False,False])
        return(res_list.loc[:,['Restaurant Name', 'count', 'avg_Rating', 'shrinkage_Rating']].head(max_sugest).reset_index(drop=True))

In [11]:
print( popularity_recommendation('Mexican', 3) )

print( popularity_recommendation('Mexican', 3, 'Shrinkage') )

  Restaurant Name  count  avg_Rating  shrinkage_Rating
0   Fonda Cantina      6    5.000000          4.015475
1          Zentli     17    4.764706          4.192601
2     Taco Diablo     37    4.297297          4.092667
  Restaurant Name  count  avg_Rating  shrinkage_Rating
0          Zentli     17    4.764706          4.192601
1     Taco Diablo     37    4.297297          4.092667
2   Fonda Cantina      6    5.000000          4.015475
